In [4]:
from data.load import load_training_data

file_path = "/Users/sanch/Downloads/training.json"

df = load_training_data("/Users/sanch/Downloads/training.json")
df.head()

,name,versions,n_versions,n_signed_version_count,latest_version,t_first_release,t_age_first_release_days,t_last_release,t_age_last_release_days,has_single_release,...,licenses,latest_summary,latest_description,distinct_authors,distinct_maintainers,distinct_keywords,distinct_classifiers,n_latest_dependency_count,latest_dependencies,is_spam
0,alabaster,"0.6.3,0.1.0,0.2.0,0.3.1,0.6.1,0.6.2,0.4.0,0.4....",29,6,1.0.0,2014-01-10 22:47:41.679631 UTC,4324,2024-07-26 18:15:03.762569 UTC,474,False,...,UNKNOWN,"A light, configurable Sphinx theme",.. image:: https://img.shields.io/pypi/v/alaba...,"[Jeff Forcier <jeff@bitprophet.org>, jeff@bitp...",[Jeff Forcier <jeff@bitprophet.org>],[],"[Programming Language :: Python :: 3.11, Topic...",0,[],0
1,cloudformation-cli-python-plugin,"2.1.10,2.1.10,2.1.4,2.1.1,2.1.8,2.1.8,2.1.7,2....",12,0,2.1.10,2020-06-29 21:41:49.945446 UTC,1962,2025-03-31 18:42:19.846927 UTC,226,False,...,Apache License 2.0,Python 3.6 and 3.7 language support for the Cl...,# AWS CloudFormation Resource Provider Python ...,[aws-cloudformation-developers@amazon.com],[],[Amazon Web Services AWS CloudFormation],"[Programming Language :: Python :: 3.6, Topic ...",0,[],0
2,drf-nested-routers,"0.1.0,0.1.1,0.1.2,0.1.3,0.9.0,0.93.4,0.93.3,0....",24,0,0.95.0,2013-11-08 17:02:22.080422 UTC,4387,2025-09-09 02:01:58.201943 UTC,64,False,...,Apache,Nested resources for the Django Rest Framework,"**This is a work in progress. It ""works for me...","[alan.justino@yahoo.com.br, alan.justino@yahoo...",[],[],"[Programming Language :: Python :: 3.5, Framew...",2,"[djangorestframework>=3.15.0, Django>=4.2]",0
3,fqdn,"1.1.0,1.2.0,1.5.1,1.4.0,1.2.0,1.5.1,1.4.0,0.0....",11,0,1.5.1,2017-06-06 15:54:10.421193 UTC,3081,2021-03-11 07:16:29.080322 UTC,1707,False,...,MPL 2.0,Validates fully-qualified domain names against...,NaN,"[ypcrts@users.noreply.github.com, 0x6768@gmail...",[],"[fqdn,domain,hostname,RFC3686,dns, fqdn]","[Programming Language :: Python :: 3.5, Progra...",0,[],0
4,mkdocs-section-index,"0.3.10,0.3.1,0.3.6,0.3.6,0.3.2,0.3.5,0.3.7,0.3...",16,0,0.3.10,2020-11-20 00:43:32.563465 UTC,1818,2025-04-05 20:56:45.387447 UTC,221,False,...,MIT,MkDocs plugin to allow clickable sections that...,# mkdocs-section-index **[Plugin][] for [MkDo...,"[Oleh Prypin <oleh@pryp.in>, oleh@pryp.in]",[],"[mkdocs, mkdocs-plugin, mkdocs,mkdocs-plugin]","[Programming Language :: Python :: 3.9, Enviro...",1,[mkdocs>=1.2],0


In [5]:
print(df.columns)

Index(['name', 'versions', 'n_versions', 'n_signed_version_count',
       'latest_version', 't_first_release', 't_age_first_release_days',
       't_last_release', 't_age_last_release_days', 'has_single_release',
       't_time_of_day_bucket', 'last_upload', 'has_homepage', 'has_repo_url',
       'cat_repo_host', 'has_issue_tracker', 'has_docs_url', 'has_license',
       'latest_project_urls', 'licenses', 'latest_summary',
       'latest_description', 'distinct_authors', 'distinct_maintainers',
       'distinct_keywords', 'distinct_classifiers',
       'n_latest_dependency_count', 'latest_dependencies', 'is_spam'],
      dtype='object')
